In [1]:
import openai
import numpy as np
import json
import os

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_KEY

In [44]:
chat_lm_id = "gpt-3.5-turbo"
translation_lm_id = "text-embedding-ada-002"
system_message="""
You are a large language model for planning a robot's movements to achieve a given task. 
The user will provide you with a task and you will instruct the user (a robot) to perform the task.
The robot may move to an object, pick it up, or place it in a new location.
The robot may also move to a location, and place an object there.
Give a command in the form of "move to <object> and pick up <object> and place it in <location>".
"""
original_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "Take out the trash."},
    {"role": "assistant", "content": """
    1. Move to the trash can
    2. Pick up the trash
    3. Move to the dumpster
    4. Place the trash in the dumpster
    """},
    {"role": "user", "content": "Make a cup of cofee."},
]

In [45]:
res = openai.ChatCompletion.create(
    model=chat_lm_id,
    messages=original_messages
)

In [46]:
res_message = res["choices"][0]["message"]

In [50]:
original_messages.append(res_message.to_dict())
original_messages.append({
    "role": "user",
    "content": "Where can i find the trash can. I am currently in the bathroom."
})

In [51]:
res2 = openai.ChatCompletion.create(
    model=chat_lm_id,
    messages=original_messages
)

In [52]:
res2

<OpenAIObject chat.completion id=chatcmpl-6u1JYYc63PTxVexfXPoAR2M312RFm at 0x7fb33416d2b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I apologize for not understanding your question earlier. Here are the instructions to help you locate the trash can:\n\n    1. Look around the bathroom to see if there is a trash can visible.\n    2. If you cannot locate the trash can, try checking other rooms in your house as you may have a separate trash can located in another room.\n    3. If you still cannot locate the trash can, ask someone else in your home for assistance in finding it.\n    4. Once you have located the trash can, you can dispose of your waste and follow the previous instructions I gave you to take out the trash.",
        "role": "assistant"
      }
    }
  ],
  "created": 1678809440,
  "id": "chatcmpl-6u1JYYc63PTxVexfXPoAR2M312RFm",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": 

In [38]:
om.update("hello")

ValueError: dictionary update sequence element #0 has length 1; 2 is required